In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [14]:
meta_data = pd.read_csv(r"C:\Users\inadnl\OneDrive - ABB\Adarsh work\Dissonance\Data\vsb-data\vsb-power-line-fault-detection\metadata_train.csv")
meta_data.head()
print(f'Number of unique labels in the dataset are {np.unique(meta_data["target"])}')

Number of unique labels in the dataset are [0 1]


In [10]:
data = pd.read_pickle(r"C:\Users\inadnl\OneDrive - ABB\Adarsh work\Dissonance\Data\vsb-data\vsb-power-line-fault-detection\train_peaks.pkl")
data.head()

,px,height,ratio_next,ratio_prev,small_dist_to_min,sawtooth_rmse,signal_id,phase_50hz,phase_aligned_x,Q
0,24554.0,4.705156,0.272445,0.902694,-1.0,0.080285,0,0.565718,133.462535,1
1,26087.0,5.568916,0.254407,0.785581,-1.0,0.081591,0,0.565718,134.152385,1
2,31681.0,8.415760,0.068728,0.891094,-1.0,0.075479,0,0.565718,136.669685,1
3,36913.0,6.569330,0.667703,0.816569,-1.0,0.099316,0,0.565718,139.024085,1
4,46340.0,5.765833,0.898712,0.897689,1.0,0.100638,0,0.565718,143.266235,1


In [12]:
print(data.shape)
print(f'Number of unique labels in the dataset are {np.unique(data["Q"])}')

(1355697, 10)
Number of unique labels in the dataset are [0 1 2 3]


In [8]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

def preprocess_vsb(data):
    # Separate features and target variable
    X = data.drop('Q', axis=1)
    y = data['Q']

    # Split the data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Standardize the features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    return X_train_scaled, y_train, X_test_scaled, y_test


In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Separate features and target variable
X = data.drop('Q', axis=1)
y = data['Q']

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the model architecture
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_scaled, y_train, epochs=10, batch_size=128, validation_data=(X_test_scaled, y_test))


Epoch 1/10
8474/8474 [==============================] - 24s 3ms/step - loss: -1008281664.0000 - accuracy: 0.4106 - val_loss: -4178534912.0000 - val_accuracy: 0.4302
Epoch 2/10
8474/8474 [==============================] - 23s 3ms/step - loss: -17206452224.0000 - accuracy: 0.4332 - val_loss: -38534942720.0000 - val_accuracy: 0.4344
Epoch 3/10
8474/8474 [==============================] - 23s 3ms/step - loss: -84786905088.0000 - accuracy: 0.4360 - val_loss: -148478459904.0000 - val_accuracy: 0.4364
Epoch 4/10
8474/8474 [==============================] - 23s 3ms/step - loss: -258016165888.0000 - accuracy: 0.4374 - val_loss: -396956794880.0000 - val_accuracy: 0.4372
Epoch 5/10
8474/8474 [==============================] - 23s 3ms/step - loss: -607889129472.0000 - accuracy: 0.4382 - val_loss: -863412813824.0000 - val_accuracy: 0.4379
Epoch 6/10
8474/8474 [==============================] - 23s 3ms/step - loss: -1222308528128.0000 - accuracy: 0.4388 - val_loss: -1643558862848.0000 - val_accuracy

In [9]:
# Select a random sample from X_test
random_index = np.random.randint(0, len(X_test))
sample = X_test_scaled[random_index]
true_label = y_test.iloc[random_index]

# Predict the class label for the sample
predicted_prob = model.predict(np.expand_dims(sample, axis=0))
predicted_label = 1 if predicted_prob > 0.5 else 0

# Print the predicted label and the true label
print("Predicted Label:", predicted_label)
print("True Label:", true_label)


1/1 [==============================] - 0s 166ms/step
Predicted Label: 1
True Label: 3


In [ ]:
def model_for_vsb_data():
    